# Experiment 10 - Information Retrieval

# 1.2 Procedure
For the purpose of this lab, we will use Whoosh since it is a fast, featureful full-text
indexing and searching library implemented in pure Python. It will suffice for this lab,
but keep in mind that there are more suitable and better options for larger projects.


## 1.2.1 Installation

In [1]:
!pip install whoosh

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 1.5 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [3]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 439.9 kB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 880.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 676.7 kB/s eta 0:00:00 0:00:01
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105787 sha256=a443f185f765da090dfdc8cce2c422562690c9f95911c60f0204ab784d8f03f2
  Stored in directory: /home/faris/.cache/pip/wheels/9f/af/22/bf406f913dc7506a485e60dce8143741abd0a92a19337d83a3
Successfully built kaggle

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [4]:
!kaggle datasets download -d stackoverflow/stacksample
!unzip stacksample.zip

Dataset URL: https://www.kaggle.com/datasets/stackoverflow/stacksample
License(s): other
100%|█████████████████████████████████████▉| 1.11G/1.11G [03:48<00:00, 11.9MB/s]
100%|██████████████████████████████████████| 1.11G/1.11G [03:48<00:00, 5.20MB/s]
Archive:  stacksample.zip
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [6]:
import pandas as pd
questions=pd.read_csv("Questions.csv", nrows=20000)
questions

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...
...,...,...,...,...,...,...,...
19995,1114470,82266.0,2009-07-11T19:37:06Z,NaN,0,"Trim all chars off file name after first ""_""",<p>I'd like to trim these purchase order file ...
19996,1114540,2288585.0,2009-07-11T20:16:06Z,NaN,7,Xcode question: Quickly jump to a particular s...,<p>What is the quickest way to jump to a parti...
19997,1114550,131128.0,2009-07-11T20:20:11Z,NaN,3,Serializing a generic collection with XMLSeria...,<p>Why won't XMLSerializer process my generic ...
19998,1114580,87271.0,2009-07-11T20:35:46Z,NaN,1,Using Yahoo Fire Eagle on Grails / Java,<p>Has anyone implemented the Yahoo Fire Eagle...


In [8]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            20000 non-null  int64  
 1   OwnerUserId   17583 non-null  float64
 2   CreationDate  20000 non-null  object 
 3   ClosedDate    1033 non-null   object 
 4   Score         20000 non-null  int64  
 5   Title         20000 non-null  object 
 6   Body          20000 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 1.1+ MB


## 1.2.3 The Index and Schema objects
Getting started with Whoosh involves creating an index object. When creating an
index for the first time, defining its schema is necessary. The schema outlines the
fields within the index, representing different pieces of information for each document.
These fields can include the document’s title or its textual content. Each field can be
indexed for searchability and/or stored, ensuring that the indexed value is returned
with the search results, which is particularly useful for fields like the title.
Let’s start by designing the schema for our index.

In [ ]:
from whoosh.fields import Schema, TEXT, ID
# Defining index schema
schema = Schema(Id=ID(stored=True), Title=TEXT(stored=True), Body=TEXT(stored=True)) # Defining index schema

Here we choose 3 columns form the questions dataframe to be used in our index:
“Id”, “Title” and “Body” for keyword based search.
Now let’s create our index. To index documents we need define folder where to
save needed files.

In [11]:
import os.path
index_dir = "indexdir"

if not os.path.exists(index_dir):
    os.mkdir(index_dir)

Then we can simply create an index and add documents to be indexed

In [12]:
from whoosh.index import create_in
from whoosh.index import open_dir

# Creating the index
ix = create_in(index_dir, schema)

# Open the index writer
writer = ix.writer()
# Iterate over the DataFrame and add documents to the index
# we have indexed title, title_body and doc_id

for index, row in questions.iterrows():
    writer.add_document(Id=str(row['Id']), Title = row['Title']
    ,Body=row['Body'])
# Commit and close the writer

writer.commit()

## 1.2.4 How to search
Once you’ve created an index and added documents to it, you can search for those
documents. The Searcher object is the main high-level interface for reading the
index. It has lots of useful methods for getting information about the index, how-
ever, the most important method on the Searcher object is search(), which takes a
whoosh.query.Query object and returns a Results object.
Normally the list of result documents is sorted by score. The whoosh.scoring mod-
ule contains implementations of various scoring algorithms. You can set the scoring
object to use when you create the searcher using the weighting keyword argument.
The following code search the index we created and rank the results based on the
Term Frequency-Inverse Document Frequency (TF-IDF) score.

In [13]:
from whoosh.qparser import QueryParser
from whoosh.scoring import TF_IDF
from whoosh import scoring

# create the query parser
qp = QueryParser("Title", schema=schema)

# parse the query
query_sentence = "How to install"
query = qp.parse(query_sentence)
# create a searcher object
searcher_tfidf = ix.searcher(weighting=scoring.TF_IDF())
# search documents and store them
# we are returing top 3 documents
results_tfidf = searcher_tfidf.search(query, limit=3, scored=True)
# print the documents
for hit in results_tfidf:
    print(hit["Id"])
    print('\n')
    print(hit["Title"])
    print('\n')
    print('------------------\n')

102850


How can I install CPAN modules locally without root access (DynaLoader.pm line 229 error)?


------------------

145900


How can I determine that Windows Installer is performing an upgrade rather than a first time install?


------------------

351640


How to install Hibernate Tools in Eclipse?


------------------



Task 1: Test the previous search code with different queries. For each one check how
many matched results are returned.

In [19]:
from whoosh.qparser import QueryParser
from whoosh.scoring import TF_IDF
from whoosh import scoring

# create the query parser
qp = QueryParser("Title", schema=schema)

# parse the query
query_sentencees = ["How to install", "how to code in C", "python"]


query = qp.parse(query_sentence)
# create a searcher object
searcher_tfidf = ix.searcher(weighting=scoring.TF_IDF())
# search documents and store them

for sentence in query_sentencees:
    i = 0
    query = qp.parse(sentence)
    results_tfidf = searcher_tfidf.search(query, scored=True)
    print('Query:', sentence)
    for hit in results_tfidf:
        print(f"result number {i}")
        i+=1
        print(hit["Id"])
        print('\n')
        print(hit["Title"])
        print('\n')
        print('------------------\n')


Query: How to install
result number 0
102850


How can I install CPAN modules locally without root access (DynaLoader.pm line 229 error)?


------------------

result number 1
145900


How can I determine that Windows Installer is performing an upgrade rather than a first time install?


------------------

result number 2
351640


How to install Hibernate Tools in Eclipse?


------------------

result number 3
358700


How to install a windows service programmatically in C#?


------------------

result number 4
396580


How to make Firefox extension auto install in nav bar?


------------------

result number 5
398850


How to install JScience library in Eclipse?


------------------

result number 6
442230


How to manually install an artifact in Maven 2?


------------------

result number 7
463790


How do you manage your run once sql install scripts in subversion?


------------------

result number 8
540640


How can I install a CPAN module into a local directory?


------------

After removing the limit which was 3, we can notice at most 10 results are returned for each query, but if there are less than 10 results the code will return all of them.


Task 2: Repeat the previous search using the BM25F scoring algorithm, which is used
in probabilistic retrieval model. Do you see any difference in the returned results?

In [20]:
from whoosh.qparser import QueryParser
from whoosh.scoring import TF_IDF
from whoosh import scoring

# create the query parser
qp = QueryParser("Title", schema=schema)

# parse the query
query_sentencees = ["How to install", "how to code in C", "python"]


query = qp.parse(query_sentence)
# create a searcher object
searcher_tfidf = ix.searcher(weighting=scoring.BM25F())
# search documents and store them

for sentence in query_sentencees:
    i = 0
    query = qp.parse(sentence)
    results_tfidf = searcher_tfidf.search(query, scored=True)
    print('Query:', sentence)
    for hit in results_tfidf:
        print(f"result number {i}")
        i+=1
        print(hit["Id"])
        print('\n')
        print(hit["Title"])
        print('\n')
        print('------------------\n')


Query: How to install
result number 0
921780


How to install ImageMagick on MAMP?


------------------

result number 1
998260


How do you install JDK?


------------------

result number 2
351640


How to install Hibernate Tools in Eclipse?


------------------

result number 3
358700


How to install a windows service programmatically in C#?


------------------

result number 4
398850


How to install JScience library in Eclipse?


------------------

result number 5
442230


How to manually install an artifact in Maven 2?


------------------

result number 6
718040


How to install SimpleJson Package for Python


------------------

result number 7
657040


How to install VeriSign's Intermediate CA Certificates?


------------------

result number 8
751000


How to install program shortcuts for all users?


------------------

result number 9
1023710


How can I install and use ack library on Windows?


------------------

Query: how to code in C
result number 0
797720


How to 

Both outputs are relevant to the topic, but BM25, an evolution from TF-IDF, offers enhanced capabilities in handling complex scenarios. It excels in providing accurate rankings and relevance scores for longer documents, a feat where TF-IDF falls short.

## 1.2.5 Query expansion
query expansion involves evaluating a user’s input (what words were typed into the
search query area, and sometimes other types of data) and expanding the search
query to match additional documents. Query expansion involves techniques such as:
1. Linguistic query expansion such as finding synonyms of words, and searching
for the synonyms as well
2. Corpus-based query expansion, by searching a single query term at a time and
counting the most common words in the top returned documents. Repeating
this process for all query terms will give you a list of terms that co-occur fre-
quently with your query terms. Thus, they can be used as to expand the query
in the domain of that corpus.
3. Pseudo-relevance feedback, by expanding the original query with the most fre-
quent terms of the top retrieved documents.
Whoosh provides methods for computing the “key terms” of a set of documents.
For these methods, “key terms” basically means terms that are frequent in the given
documents, but relatively infrequent in the indexed collection as a whole. These
methods can be useful for query expansion.
For example, the following code retrieve more results similar to the first returned
item in the previous example

In [27]:
from whoosh.qparser import QueryParser
from whoosh.scoring import TF_IDF
from whoosh import scoring

# create the query parser
qp = QueryParser("Title", schema=schema)

# parse the query
query_sentence = "How to install"
query = qp.parse(query_sentence)
# create a searcher object
searcher_tfidf = ix.searcher(weighting=scoring.TF_IDF())
# search documents and store them
# we are returing top 3 documents
results_tfidf = searcher_tfidf.search(query, limit=3, scored=True)
# print the documents
for hit in results_tfidf:
    print(hit["Id"])
    print('\n')
    print(hit["Title"])
    print('\n')
    print('------------------\n')

102850


How can I install CPAN modules locally without root access (DynaLoader.pm line 229 error)?


------------------

145900


How can I determine that Windows Installer is performing an upgrade rather than a first time install?


------------------

351640


How to install Hibernate Tools in Eclipse?


------------------



In [28]:
more_results = results_tfidf[0].more_like_this("Title")
for hit in more_results:
    print(hit["Id"])
    print('\n')
    print(hit["Title"])
    print('\n')
    print('------------------\n')

459590


What is the difference betwen including modules and embedding modules?


------------------

423330


Why can't DynaLoader.pm load SSleay.dll for Net::SSLeay and Crypt::SSLeay?


------------------

540640


How can I install a CPAN module into a local directory?


------------------

172040


How do you develop against OpenID locally


------------------

566290


Silverlight Development - Service URL while developing locally


------------------

766830


How can I locally manage C manuals?


------------------

799860


Using Mercurial locally, only with Subversion server


------------------

852280


Ubuntu: "Could not find rails locally or in a repository"


------------------

78900


How to check for memory leaks in Guile extension modules?


------------------

199180


Is there any way to get python omnicomplete to work with non-system modules in vim?


------------------



We can also extract keywords for the top N documents in a whoosh.searching.Results
object. This requires that the field is either vectored or stored. For example, to ex-
tract five key terms from the Title field of the top ten documents of the results object
in the previous example:

In [29]:
keywords = [keyword for keyword, score
in results_tfidf.key_terms("Title", docs=10, numterms=5)]
keywords

['install', '229', 'cpan', 'dynaloader.pm', 'locally']

## 1.2.6 Evaluating IR systems
There are several measures for evaluating IR systems, such as precision, recall and
mean average precision (mAP). While precision and recall does not take the rank
of the retrieved documents into consideration, mAP considers the order in which
documents are ranked. Sometimes precision and recall are computed at cut-off value
k of retrieved documents. In this case, it is called precision@k and recall@k, which
means precision and recall computed when considering the first k retrieved documents
only.

In this part we will evaluate our IR system on a toy dataset. This dataset contains
a set of documents, set of queries, and list of relevant documents for each query.

In [31]:
queries = {
'q1': "machine learning",
'q2':"AI algorithms"
}
relevance = {
'q1' : ["doc1", "doc2", "doc3"],
'q2' : ["doc1", "doc2", "doc3", "doc4", "doc5"]
}
documents = {
'doc1': "Artificial Intelligence (AI) is transforming various industries through automation and advanced algorithms. Machine learning, a subset of AI, enables computers to learn from data and make predictions. Algorithms are at the core of AI systems, guiding decision-making and problem-solving processes. AI-powered systems are increasingly used in healthcare for diagnosis and treatment planning. The ethical implications of AI algorithms, such as bias and fairness, are important considerations in their development.",

'doc2': "Deep learning, a branch of machine learning, uses neural networks to process complex data. AI algorithms are capable of analyzing large datasets to extract meaningful insights. Natural Language Processing (NLP) algorithms enable computers to understand and generate human language. AI-driven recommendation algorithms personalize user experiences in e-commerce and content platforms. Ensuring the transparency and accountability of AI algorithms is essential for building trust in AI technologies.",

'doc3': "Reinforcement learning algorithms enable AI agents to learn through trial and error interactions with their environment. AI algorithms are used in financial markets for high-frequency trading and risk management. Computer vision algorithms enable machines to interpret and analyze visual information. AI algorithms can enhance cybersecurity by detecting and mitigating cyber threats in real-time. Continuous research and development are essential for advancing AI algorithms and overcoming their limitations.",
    
'doc4': "Evolutionary algorithms, inspired by natural selection, are used to optimize complex systems and processes. AI algorithms play a crucial role in autonomous vehicles for navigation and decision-making. Quantum computing algorithms have the potential to revolutionize AI by solving complex problems exponentially faster. AI algorithms are employed in predictive maintenance to anticipate equipment failures and reduce downtime. Ethical guidelines and regulations are needed to govern the development and deployment of AI algorithms.",
'doc5': "Genetic algorithms are used to evolve solutions to optimization and search problems inspired by natural selection. AI algorithms enable personalized content recommendations in streaming services and social media platforms. Swarm intelligence algorithms mimic the collective behavior of social insects to solve optimization problems. AI algorithms are used in drug discovery to accelerate the identification of potential treatments. Collaborative efforts are essential for advancing AI algorithms and harnessing their full potential for societal benefit."
}


Let’s start by creating an index for this dataset

In [40]:
writer.commit()

writer was locked, so needed wto commit first

In [49]:
from whoosh.qparser import QueryParser
from whoosh.scoring import TF_IDF
from whoosh import scoring
# create the query parser
qp = QueryParser("Body", schema=schema)
# parse the query
query_sentence = queries['q1']
query = qp.parse(query_sentence)
# create a searcher object
searcher_tfidf = ix.searcher(weighting=scoring.TF_IDF())
# search documents and store them
# we are returing top 3 documents
results_tfidf = searcher_tfidf.search(query, scored=True)
# print the documents
for hit in results_tfidf:
    print(hit["Id"])
    print('\n')
    print(hit["Body"])
    print('\n')
    print('------------------\n')

doc2


Deep learning, a branch of machine learning, uses neural networks to process complex data. AI algorithms are capable of analyzing large datasets to extract meaningful insights. Natural Language Processing (NLP) algorithms enable computers to understand and generate human language. AI-driven recommendation algorithms personalize user experiences in e-commerce and content platforms. Ensuring the transparency and accountability of AI algorithms is essential for building trust in AI technologies.


------------------

doc1


Artificial Intelligence (AI) is transforming various industries through automation and advanced algorithms. Machine learning, a subset of AI, enables computers to learn from data and make predictions. Algorithms are at the core of AI systems, guiding decision-making and problem-solving processes. AI-powered systems are increasingly used in healthcare for diagnosis and treatment planning. The ethical implications of AI algorithms, such as bias and fairness, are i

Task 3: Compute the precision and recall for the retrieved documents in the previous
example.

Task 4: Modify the last code to test all queries and then report the precision and
recall

In [56]:
from whoosh.qparser import QueryParser
from whoosh.scoring import TF_IDF
from whoosh import scoring

ground_truth = relevance

# Function to perform search and return results
def perform_search(searcher, query_sentence):
    query = qp.parse(query_sentence)
    results = searcher.search(query, scored=True)
    return [hit["Id"] for hit in results]

# Function to calculate precision and recall
def calculate_metrics(retrieved, relevant):
    retrieved_set = set(retrieved)
    relevant_set = set(relevant)
    true_positives = len(retrieved_set & relevant_set)
    precision = true_positives / len(retrieved) if retrieved else 0
    recall = true_positives / len(relevant) if relevant else 0
    return precision, recall

# Perform search and evaluate results
def evaluate_algorithm(searcher, algorithm_name):
    print('Task 4')
    print('------------------\n')

    for query_id, relevant_docs in ground_truth.items():
        
        query_sentence = queries[query_id]
        retrieved_docs = perform_search(searcher, query_sentence)
        precision, recall = calculate_metrics(retrieved_docs, relevant_docs)
        if query_sentence == queries['q1']:
            print('Task 3')

        print(f"Algorithm: {algorithm_name}")
        print(f"Query: {query_sentence}")
        print(f"Precision: {precision:.2f}, Recall: {recall:.2f}")
        print('------------------\n')

# Create the query parser
qp = QueryParser("Body", schema=schema)

# Perform search using TF-IDF
searcher_tfidf = ix.searcher(weighting=scoring.TF_IDF())
evaluate_algorithm(searcher_tfidf, "TF-IDF")

Task 4
------------------

Task 3
Algorithm: TF-IDF
Query: machine learning
Precision: 1.00, Recall: 0.67
------------------

Algorithm: TF-IDF
Query: AI algorithms
Precision: 1.00, Recall: 1.00
------------------



we can notice that the score of the second query is higher than the first one, which is expected since the second query is more general than the first one, so algorithim returend all the right ones, while the first query only 2 out of 3 were returned. Both tasks has been shortened to one code block because basically they are the same, only the query differs